In [50]:
# Importing libraries
import matplotlib.pyplot as plt
import scipy
import numpy as np
import keras.utils
import tensorflow as tf
import torch
from torch.optim import SGD
import time
from LeNet5_NEW import LeNet5
import os
from LettersIsolation import isolate_letters

# Retrieving data
PATH = os.path.join(os.getcwd(), "matlab", "emnist-letters.mat")
EMNIST = scipy.io.loadmat(PATH)
x_train = EMNIST["dataset"][0][0][0][0][0][0].astype("float64")
y_train = EMNIST["dataset"][0][0][0][0][0][1]

x_test = EMNIST['dataset'][0][0][1][0][0][0].astype("float64")
y_test = EMNIST['dataset'][0][0][1][0][0][1]

# Filter out lowercase letters (class labels 1 to 26)
lowercase_indices_train = np.where((y_train >= 1) & (y_train <= 26))[0]
lowercase_indices_test = np.where((y_test >= 1) & (y_test <= 26))[0]

x_train = x_train[lowercase_indices_train]
y_train = y_train[lowercase_indices_train]

x_test = x_test[lowercase_indices_test]
y_test = y_test[lowercase_indices_test]

# Scaling data
x_train = (x_train - np.mean(x_train)) / np.std(x_train)
x_test = (x_test - np.mean(x_train)) / np.std(x_train)

nb_classes = 26  # Number of classes

y_train = keras.utils.to_categorical(y_train-1, nb_classes)
y_test = keras.utils.to_categorical(y_test-1, nb_classes)

x_train_scaled = x_train.reshape(-1, 28, 28, 1)
x_test_scaled = x_test.reshape(-1, 28, 28, 1)


# Padding to have 32x32 images has in the paper about LeNet5
x_train_padded = np.array(tf.pad(tensor=x_train_scaled, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]]))
x_test_padded = np.array(tf.pad(tensor=x_test_scaled, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]]))



# Pytorch needs a special format
x_train_tensor = torch.tensor(x_train_padded, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test_padded, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

start = time.time() # Measuring computation time

num_epochs = 15
batch_size = 200

# Model of CNN
model = LeNet5(nb_classes)
# Loss
criterion = torch.nn.CrossEntropyLoss()

# Optimizer
optimizer = SGD(model.parameters(), lr=0.05)

# Training
for epoch in range(num_epochs):
    # Shuffle the data at the beginning of an epoch

    indices = np.arange(len(x_train_tensor))
    np.random.shuffle(indices)
    total_correct_predictions = 0
    total_samples = 0

    for i in range(0, len(x_train), batch_size):
        # Select batch by batch
        batch_indices = indices[i:i + batch_size]
        inputs = x_train_tensor[batch_indices].permute(0, 3, 1, 2)

        labels = y_train_tensor[batch_indices]

        #Forward propagation
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        with torch.no_grad():
            model.eval()
            _, predicted_labels = torch.max(outputs, 1)
            _, target_labels = torch.max(labels, 1)
            batch_correct_predictions = (predicted_labels == target_labels).sum().item()

            # Accumulate the total number of correct predictions and total samples
            total_correct_predictions += batch_correct_predictions
            total_samples += labels.size(0)
            model.train()

        # Back-propagation and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate accuracy at the end of the epoch
    epoch_accuracy = total_correct_predictions / total_samples

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {epoch_accuracy:.4f}')




Epoch [1/15], Loss: 0.9490, Accuracy: 0.4089
Epoch [2/15], Loss: 0.7446, Accuracy: 0.7273
Epoch [3/15], Loss: 0.5558, Accuracy: 0.7994
Epoch [4/15], Loss: 0.4480, Accuracy: 0.8362
Epoch [5/15], Loss: 0.4028, Accuracy: 0.8550
Epoch [6/15], Loss: 0.2910, Accuracy: 0.8671
Epoch [7/15], Loss: 0.2826, Accuracy: 0.8764
Epoch [8/15], Loss: 0.4351, Accuracy: 0.8824
Epoch [9/15], Loss: 0.3477, Accuracy: 0.8877
Epoch [10/15], Loss: 0.2612, Accuracy: 0.8917
Epoch [11/15], Loss: 0.2371, Accuracy: 0.8961
Epoch [12/15], Loss: 0.2922, Accuracy: 0.8979
Epoch [13/15], Loss: 0.2441, Accuracy: 0.9010
Epoch [14/15], Loss: 0.2017, Accuracy: 0.9047
Epoch [15/15], Loss: 0.2770, Accuracy: 0.9056


In [51]:
torch.save(model, os.path.join(os.getcwd(), "model.pt" ))

In [52]:
model = torch.load(os.path.join(os.getcwd(), "model.pt"))

# Evaluation loop
inputs = x_test_tensor.permute(0, 3, 1, 2)
labels = y_test_tensor
total_correct_predictions = 0
total_samples = 0
# Forward propagation
model.eval()
outputs = model(inputs)

# Compute the loss
loss_test = criterion(outputs, labels)
# Set model to evaluation mode and evaluate it
with torch.no_grad():
    model.eval()
    _, predicted_labels = torch.max(outputs, 1)
    _, target_labels = torch.max(labels, 1)
    batch_correct_predictions = (predicted_labels == target_labels).sum().item()

# Accumulate the total number of correct predictions and total samples
total_correct_predictions += batch_correct_predictions
total_samples += labels.size(0)

accuracy_test = total_correct_predictions / total_samples

print(f'TEST : Loss: {loss_test.item():.4f}, Accuracy: {accuracy_test:.4f}')

# Affichage de quelques résultats
#for i in range(20):
#    print('Essai', i, ':\n Predicted =', predicted_labels[i], '\n Correction =', target_labels[i],'\n')




TEST : Loss: 9.6605, Accuracy: 0.9080


In [1]:
import torch
import os
from LettersIsolation import isolate_letters
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

model = torch.load(os.path.join(os.getcwd(), "model.pt"))


filepath = os.path.join("C:/Users", "msii", "Downloads", "yasQueen.jpg")
print(filepath)

mot =""
for letter in isolate_letters(filepath):

    letter = np.expand_dims(letter, axis=-1)

    # Ajouter une dimension pour le batch, tranformation en torch.tensor et permutation des index
    letter = torch.tensor(np.expand_dims(letter, axis=0), dtype = torch.float32).permute(0, 3, 1, 2)

    outputs = model(letter)
    mot += chr(np.argmax(outputs.detach().numpy()[0]) + 97) # Convertir la prédiction en lettre minuscule

print(mot)

C:/Users\msii\Downloads\yasQueen.jpg
yagqueen
